$$ \huge \color{blue}{\text {Derivation of Euler equations in primitive form and preconditioning }}$$


$$ \large \color{blue}{\text{In this notebook we will do the derivation of the Weiss and Smith preconditioning.}}$$

In [1]:
kill(all);

(%o0)                                done

# Analysis of the Euler Equations
### 1. conservative form


    

### We want to solve the Euler equations, which is the system of PDE's:

$$ \color {blue} {\large \frac{\partial U}{\partial t} + \frac{\partial F(U)}{\partial x} + \frac{\partial G(U)}{\partial y} = 0}$$


    
### For the 2D problem we have the following definitions:


$$ \color{blue}{\large
U = \left\{
    \begin{array}\\
        \rho               &  \\
        \rho u        &  \\
        \rho v        &  \\
        \rho E u & 
    \end{array}
\right\}}
$$


$$ \color{blue}{\large
F = \left\{
    \begin{array}\\
        \rho  u                  &  \\
        p+\rho u^2               &  \\
        \rho  u  v          &  \\
        (\rho E + p) u & 
    \end{array}
\right\}}
$$


$$ \color{blue}{\large
G = \left\{
    \begin{array}\\
        \rho v & \\
        \rho  u  v          &  \\
        p+\rho v^2               &  \\
        (\rho E + p) u & 
    \end{array}
\right\}}
$$



### For the energy we have the following relationship between internal energy $e$ and total energy $E$:
$$\color{blue}{\large E = e + q,}$$

### with $q = \frac{1}{2}(u^2+v^2)$. The total energy is related to the total enthalpy $H$ by

$$\color{blue}{\large E = H - \frac{p}{\rho}.}$$

### The total enthalpy is related to the sensible enthalpy according to:

$$\color{blue}{\large H = h + q = c_pT + q, }$$

### In a calorically perfect gas, the heat capacity is constant, and we can write the sensible enthalpy as $h=c_pT$. We have for the total energy:

$$\color{blue}{\large E = (c_pT + q) - \frac{p}{\rho}}$$


In [2]:
consSystem : [U1=rho,U2=rho*u, U3=rho*v, U4=rho*E];

F:ratsimp((matrix([rho*u,rho*u*u+p,rho*u*v,u*rho*E+p*u])));
    
G:ratsimp((matrix([rho*v,rho*u*v,rho*v*v+p,v*rho*E+p*v])));

(%o1)           [U1 = rho, U2 = rho u, U3 = rho v, U4 = E rho]

(%o2)            [             2                             ]
                 [ rho u  rho u  + p  rho u v  (E rho + p) u ]

(%o3)            [                      2                    ]
                 [ rho v  rho u v  rho v  + p  (E rho + p) v ]

### We define the velocity q as follows:

In [3]:
q : (u*u+v*v)/2;

                                     2    2
                                    v  + u
(%o4)                               -------
                                       2

In [4]:
e : E-q;

                                       2    2
                                      v  + u
(%o5)                             E - -------
                                         2

### The pressure follows from the pressure definition: $p = (\gamma - 1)\rho e$

In [5]:
p: (gamma-1)*rho*e;

                                   2    2
                                  v  + u
(%o6)                    rho (E - -------) (gamma - 1)
                                     2

### We now map the conservative variables to $(\rho,u,v,E)$

In [6]:
mapping : [rho = U1, u=U2/U1, v = U3/U1, E=U4/U1];

                                     U2      U3      U4
(%o7)                 [rho = U1, u = --, v = --, E = --]
                                     U1      U1      U1

### From this also follows the pressure in terms of the conservative variables:

In [7]:
P : subst(mapping,p);

                                   2     2
                                 U3    U2
                                 --- + ---
                                   2     2
                            U4   U1    U1
(%o8)                   U1 (-- - ---------) (gamma - 1)
                            U1       2

### We now write the convective flux as function of the conservative variables:

In [8]:
Fstar : ratexpand(ratsimp(ev(subst(mapping,F),nouns)));
transpose(ratsimp(Fstar));

(%o9)  Col 1 = [ U2 ]
         [              2           2                2        2 ]
 Col 2 = [            U3  gamma   U2  gamma        U3     3 U2  ]
         [ U4 gamma - --------- - --------- - U4 + ---- + ----- ]
         [              2 U1        2 U1           2 U1   2 U1  ]
         [ U2 U3 ]
 Col 3 = [ ----- ]
         [  U1   ]
         [                    2           3              2      3  ]
         [ U2 U4 gamma   U2 U3  gamma   U2  gamma   U2 U3     U2   ]
 Col 4 = [ ----------- - ------------ - --------- + ------ + ----- ]
         [     U1               2             2         2        2 ]
         [                  2 U1          2 U1      2 U1     2 U1  ]

            [                         U2                          ]
            [                                                     ]
            [              2     2                      2       2 ]
            [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + 3 U2  ]
            [ --------------------------------------------------- ]
            [                        2 U1                         ]
            [                                                     ]
(%o10)      [                        U2 U3                        ]
            [                        -----                        ]
            [                         U1                          ]
            [                                                     ]
            [                     2     3               2     3   ]
            [  (2 U1 U2 U4 - U2 U3  - U2 ) gamma + U2 U3  + U2    ]
            [  ------------------------------------------------   ]
            [                           2       

In [9]:
Utilde:matrix([U1,U2,U3,U4]);

(%o11)                        [ U1  U2  U3  U4 ]

In [10]:
dFdU:ratsimp(jacobian(args(Fstar)[1],args(Utilde)[1]));

                [                        0                         ]
                [                                                  ]
                [            2     2            2       2          ]
                [         (U3  + U2 ) gamma - U3  - 3 U2           ]
                [         -------------------------------          ]
                [                          2                       ]
                [                      2 U1                        ]
                [                                                  ]
(%o12)  Col 1 = [                       U2 U3                      ]
                [                     - -----                      ]
                [                          2                       ]
                [                        U1                        ]
                [                                                  ]
                [                    2     3               2     3 ]
                [   (U1 U2 U4 - U2

    
### Since $\frac{\partial F(U)}{\partial x} = \frac{\partial F}{\partial U}\frac{\partial U}{\partial x}$ we can now write the Euler equation in quasilinear form with 


$$\color{blue}{\large \frac{dU}{dt} + \frac{\partial F}{\partial U}\frac{dU}{dx} + \frac{\partial G}{\partial U}\frac{dU}{dy}= \frac{dU}{dt}+A\frac{dU}{dx}+B\frac{dU}{dx}=0.}$$
    


In [11]:
A:ratsimp(subst(consSystem,dFdU));

                [                  0                  ]
                [                                     ]
                [       2    2           2      2     ]
                [     (v  + u ) gamma - v  - 3 u      ]
                [     ---------------------------     ]
(%o13)  Col 1 = [                  2                  ]
                [                                     ]
                [                - u v                ]
                [                                     ]
                [     2    3                   2    3 ]
                [ (u v  + u  - E u) gamma - u v  - u  ]
         [                   1                   ]
         [                                       ]         [        0        ]
         [             3 u - u gamma             ]         [                 ]
         [                                       ]         [   v - v gamma   ]
 Col 2 = [                   v                   ] Col 3 = [                 ]
         [       

### We can get the internal energy from the pressure equation and write pressure as $p=\rho c^2/\gamma$:

In [12]:
pc : rho*c^2/gamma;
e : pc/(rho*(gamma-1));

                                     2
                                    c  rho
(%o14)                              ------
                                    gamma

                                       2
                                      c
(%o15)                         -----------------
                               (gamma - 1) gamma

In [13]:
A : ratsimp(subst([E = (e + q)],A));

                               2    2           2      2
                             (v  + u ) gamma - v  - 3 u
(%o16) matrix([0, 1, 0, 0], [---------------------------, 3 u - u gamma, 
                                          2
v - v gamma, gamma - 1], [- u v, v, u, 0], 
     2    3       2            2       3               2      3      2
 (u v  + u ) gamma  + ((- 3 u v ) - 3 u ) gamma + 2 u v  + 2 u  - 2 c  u
[-----------------------------------------------------------------------, 
                               2 gamma - 2
     2      2        2       2           2      2      2
  2 u  gamma  + ((- v ) - 5 u ) gamma + v  + 3 u  - 2 c
- ------------------------------------------------------, u v - u v gamma, 
                       2 gamma - 2
u gamma])

### We can now compute the eigenvalues, which should be $(u, u, u-c, u+c)$

In [14]:
ratexpand(ratsimp(eigenvalues(A)));

(%o17)                  [[u, u - c, u + c], [2, 1, 1]]

### We use the same approach for the matrix B:

In [15]:
Gstar : ratexpand(ratsimp(ev(subst(mapping,G),nouns)));
dGdU:ratsimp(jacobian(args(Gstar)[1],args(Utilde)[1]));
B:ratsimp(subst(consSystem,dGdU));
B : ratsimp(subst([E = (e + q)],B));

                               [ U2 U3 ]
(%o18)  Col 1 = [ U3 ] Col 2 = [ ----- ]
                               [  U1   ]
         [              2           2                  2     2  ]
 Col 3 = [            U3  gamma   U2  gamma        3 U3    U2   ]
         [ U4 gamma - --------- - --------- - U4 + ----- + ---- ]
         [              2 U1        2 U1           2 U1    2 U1 ]
         [                 3           2               3      2    ]
         [ U3 U4 gamma   U3  gamma   U2  U3 gamma    U3     U2  U3 ]
 Col 4 = [ ----------- - --------- - ------------ + ----- + ------ ]
         [     U1              2            2           2       2  ]
         [                 2 U1         2 U1        2 U1    2 U1   ]

                [                        0                         ]
                [                                                  ]
                [                       U2 U3                      ]
                [                     - -----                      ]
                [                          2                       ]
                [                        U1                        ]
                [                                                  ]
                [            2     2              2     2          ]
(%o19)  Col 1 = [         (U3  + U2 ) gamma - 3 U3  - U2           ]
                [         -------------------------------          ]
                [                          2                       ]
                [                      2 U1                        ]
                [                                                  ]
                [                 3     2               3     2    ]
                [   (U1 U3 U4 - U3

                [                  0                  ]
                [                                     ]
                [                - u v                ]
                [                                     ]
                [       2    2             2    2     ]
(%o20)  Col 1 = [     (v  + u ) gamma - 3 v  - u      ]
                [     ---------------------------     ]
                [                  2                  ]
                [                                     ]
                [   3     2                  3    2   ]
                [ (v  + (u  - E) v) gamma - v  - u  v ]
                                     [                   1                   ]
         [        0        ]         [                                       ]
         [                 ]         [                   u                   ]
         [        v        ]         [                                       ]
 Col 2 = [                 ] Col 3 = [             3 v - v gamma    

(%o21) matrix([0, 0, 1, 0], [- u v, v, u, 0], 
   2    2             2    2
 (v  + u ) gamma - 3 v  - u
[---------------------------, u - u gamma, 3 v - v gamma, gamma - 1], 
              2
   3    2         2          3       2               3       2      2
 (v  + u  v) gamma  + ((- 3 v ) - 3 u  v) gamma + 2 v  + (2 u  - 2 c ) v
[-----------------------------------------------------------------------, 
                               2 gamma - 2
                      2      2          2     2             2    2      2
                   2 v  gamma  + ((- 5 v ) - u ) gamma + 3 v  + u  - 2 c
u v - u v gamma, - ------------------------------------------------------, 
                                        2 gamma - 2
v gamma])

### We now have the PDE in quasi-linear form:

$$\color{blue}{\large  \frac{\partial U}{\partial t} + A \cdot \frac{\partial U}{\partial x} + B \cdot \frac{\partial U}{\partial y} = 0}$$
    


***
### We can now compute the eigenvalues, which should be $(u, u, u-c, u+c)$  
***

In [16]:
ratexpand(ratsimp(eigenvalues(A)));

(%o22)                  [[u, u - c, u + c], [2, 1, 1]]

In [17]:
ratexpand(ratsimp(eigenvalues(B)));

(%o23)                  [[v, v - c, v + c], [2, 1, 1]]

### We can write the condition number in terms of the mach number using $M = \frac{u}{c}$, so the eigenvalues become

$$\color{blue}{ \large (Mc, Mc-c, Mc+c)}$$

In [18]:
condition(M):=(M+1)/M;

                                             M + 1
(%o24)                       condition(M) := -----
                                               M

In [19]:
plot2d(condition(u),[u,0.1,1.5]);

(%o25)                               false

***
#  2. primitive form with $V=(\rho,u,v,P)$

### The euler equation is now in quasilinear form with 

$$\color{blue}{\large  \frac{\partial U}{\partial t} + A \cdot \frac{\partial U}{\partial x} + B \cdot \frac{\partial U}{\partial y} = 0}$$
    
### We now write the equations in terms of the new primitive variables $V=(\rho,u,v,p)$.


### We have the relationships:

$$\color{blue}{\large \frac{\partial U(V)}{\partial t} = \frac{\partial U}{\partial V}\frac{\partial V}{\partial t} = M \frac{\partial V}{\partial t}}$$
$$\color{blue}{\large \frac{\partial U(V)}{\partial x} = \frac{\partial U}{\partial V}\frac{\partial V}{\partial x} = M \frac{\partial V}{\partial x}}$$
    
### the Euler equations in quasilinear form can now be written as: 

$$\color{blue}{\large \frac{dU(V)}{dt} + A\frac{dU(V)}{dx}=M\frac{\partial V}{\partial t} + A M \frac{\partial V}{\partial x}=0}$$
    
### Or, when collecting the matrices:
    
$$\color{blue}{\large \frac{\partial V}{\partial t} + M^{-1} A M \frac{\partial V}{\partial x}=0}$$

    



In [20]:
/* compact representation of primitive variables rho, u, p*/

RHO : U1;
Up : U2/U1;
Vp : U3/U1;
P;

(%o26)                                U1

                                      U2
(%o27)                                --
                                      U1

                                      U3
(%o28)                                --
                                      U1

                                   2     2
                                 U3    U2
                                 --- + ---
                                   2     2
                            U4   U1    U1
(%o29)                  U1 (-- - ---------) (gamma - 1)
                            U1       2

In [21]:
Vtilde:ratsimp(matrix([RHO,Up,Vp, P]));

       [                          2     2                      2     2 ]
(%o30) [     U2  U3  (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
       [ U1  --  --  ------------------------------------------------- ]
       [     U1  U1                        2 U1                        ]

In [22]:
Utilde:matrix([U1,U2,U3,U4]);
dVdU:ratsimp(jacobian(args(Vtilde)[1],args(Utilde)[1]));
dUdV : ratsimp(invert(dVdU));
M : ratsimp(subst(consSystem,dUdV));
ratsimp(invert(M));
Atilde : ratsimp(invert(M).A.M);
ratexpand(ratsimp(eigenvalues(Atilde)));

(%o31)                        [ U1  U2  U3  U4 ]

(%o32) 
 [               1                       0                0             0     ]
 [                                                                            ]
 [               U2                     1                                     ]
 [             - ---                    --                0             0     ]
 [                 2                    U1                                    ]
 [               U1                                                           ]
 [                                                                            ]
 [               U3                                      1                    ]
 [             - ---                     0               --             0     ]
 [                 2                                     U1                   ]
 [               U1                                                           ]
 [                                                                            ]
 [    2     2            2     2

                       [     1      0   0       0     ]
                       [                              ]
                       [    U2                        ]
                       [    --      U1  0       0     ]
                       [    U1                        ]
                       [                              ]
                       [    U3                        ]
(%o33)                 [    --      0   U1      0     ]
                       [    U1                        ]
                       [                              ]
                       [   2     2                    ]
                       [ U3  + U2               1     ]
                       [ ---------  U2  U3  --------- ]
                       [       2            gamma - 1 ]
                       [   2 U1                       ]

                     [    1       0      0        0     ]
                     [                                  ]
                     [    u      rho     0        0     ]
                     [                                  ]
(%o34)               [    v       0     rho       0     ]
                     [                                  ]
                     [  2    2                          ]
                     [ v  + u                     1     ]
                     [ -------  rho u  rho v  --------- ]
                     [    2                   gamma - 1 ]

       [             1                   0            0           0     ]
       [                                                                ]
       [              u                  1                              ]
       [           - ---                ---           0           0     ]
       [             rho                rho                             ]
       [                                                                ]
(%o35) [              v                               1                 ]
       [           - ---                 0           ---          0     ]
       [             rho                             rho                ]
       [                                                                ]
       [   2    2           2    2                                      ]
       [ (v  + u ) gamma - v  - u                                       ]
       [ -------------------------  u - u gamma  v - v gamma  gamma - 1 ]
       [             2                

                             [ u   rho    0   0  ]
                             [                   ]
                             [                1  ]
                             [ 0    u     0  --- ]
(%o36)                       [               rho ]
                             [                   ]
                             [ 0    0     u   0  ]
                             [                   ]
                             [     2             ]
                             [ 0  c  rho  0   u  ]

(%o37)                  [[u - c, u + c, u], [1, 1, 2]]

### Note that this corresponds (besides some typing errors) to the matrices in the lecture notes of Jameson AA215A, lecture 4, p 10

***
# 3. primitive form with $V=(P,u,v,T)$



### We have 

$$\color{blue}{\large E = c_pT + q - \frac{p}{\rho}}$$

$$\color{blue}{\large  T = \frac{\rho E - \rho q + p }{\rho c_p}}$$

### Now write $T$ in terms of the variables $U_1=\rho$, $U_2=\rho u$, $U_3 = \rho v$, $U_4 = \rho E$.
### We already have P in terms of U:


In [23]:
Q : ((U2/U1)^2 + (U3/U1)^2)/2;
T : ratsimp((U4 - U1*Q + P)/(U1*cp));

                                     2     2
                                   U3    U2
                                   --- + ---
                                     2     2
                                   U1    U1
(%o38)                             ---------
                                       2

                                       2     2
                          (2 U1 U4 - U3  - U2 ) gamma
(%o39)                    ---------------------------
                                       2
                                   2 U1  cp

In [24]:
/* this is the only thing that needs to be changed to switch to other primitive variables*/
Vtilde:ratsimp(matrix([P,Up,Vp,T]));

                [              2     2                      2     2 ]
(%o40)  Col 1 = [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [ ------------------------------------------------- ]
                [                       2 U1                        ]
                                                [              2     2        ]
                  [ U2 ]         [ U3 ]         [ (2 U1 U4 - U3  - U2 ) gamma ]
          Col 2 = [ -- ] Col 3 = [ -- ] Col 4 = [ --------------------------- ]
                  [ U1 ]         [ U1 ]         [              2              ]
                                                [          2 U1  cp           ]

In [25]:
Utilde:matrix([U1,U2,U3,U4]);
dVdU:ratsimp(jacobian(args(Vtilde)[1],args(Utilde)[1]));
dUdV : ratsimp(invert(dVdU));
consSystem;
M : ratsimp(subst(consSystem,dUdV));
/* substitute energy*/
M : ratsimp(subst(E=e+q,M));
A;
Atilde : ratsimp(invert(M).A.M);
ratexpand(ratsimp(eigenvalues(Atilde)));

(%o41)                        [ U1  U2  U3  U4 ]

(%o42) 
 [    2     2            2     2                                              ]
 [ (U3  + U2 ) gamma - U3  - U2     U2 gamma - U2    U3 gamma - U3            ]
 [ -----------------------------  - -------------  - -------------  gamma - 1 ]
 [                 2                     U1               U1                  ]
 [             2 U1                                                           ]
 [                                                                            ]
 [               U2                     1                                     ]
 [             - ---                    --                0             0     ]
 [                 2                    U1                                    ]
 [               U1                                                           ]
 [                                                                            ]
 [               U3                                      1                    ]
 [             - ---            

                [                           2                       ]
                [                       2 U1                        ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
                [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [                                                   ]
                [                      2 U1 U2                      ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
(%o43)  Col 1 = [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [                                                   ]
                [                      2 U1 U3                      ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
                [ (2

(%o44)          [U1 = rho, U2 = rho u, U3 = rho v, U4 = E rho]

                [                     2                   ]
                [ - ------------------------------------- ]
                [     2    2                 2    2       ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]
                [                                         ]
                [                    2 u                  ]
                [ - ------------------------------------- ]         [   0   ]
                [     2    2                 2    2       ]         [       ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]         [  rho  ]
(%o45)  Col 1 = [                                         ] Col 2 = [       ]
                [                    2 v                  ]         [   0   ]
                [ - ------------------------------------- ]         [       ]
                [     2    2                 2    2       ]         [ rho u ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]
                [                 

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o46)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

                               2    2           2      2
                             (v  + u ) gamma - v  - 3 u
(%o47) matrix([0, 1, 0, 0], [---------------------------, 3 u - u gamma, 
                                          2
v - v gamma, gamma - 1], [- u v, v, u, 0], 
     2    3       2            2       3               2      3      2
 (u v  + u ) gamma  + ((- 3 u v ) - 3 u ) gamma + 2 u v  + 2 u  - 2 c  u
[-----------------------------------------------------------------------, 
                               2 gamma - 2
     2      2        2       2           2      2      2
  2 u  gamma  + ((- v ) - 5 u ) gamma + v  + 3 u  - 2 c
- ------------------------------------------------------, u v - u v gamma, 
                       2 gamma - 2
u gamma])

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o48)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

(%o49)                  [[u - c, u + c, u], [1, 1, 2]]


### Note that the PDE is now written in terms of primitive variables as:

$$\color{blue}{\large  \frac{\partial V}{\partial t} + M^{-1} A M \frac{\partial V}{\partial x}=0}$$


$$\color{blue}{\large \frac{\partial V}{\partial t} + \tilde A \frac{\partial V}{\partial x}=0}$$
    


# 3.1 preconditioning

### We can use a preconditioner matrix in front of the temporal derivative to influence the eigenvalues. The steady state solution will still be the same. Example:
    


In [26]:
Gamma : matrix([theta,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]);

                              [ theta  0  0  0 ]
                              [                ]
                              [   0    1  0  0 ]
(%o50)                        [                ]
                              [   0    0  1  0 ]
                              [                ]
                              [   0    0  0  1 ]

In [27]:
invert(Gamma).Atilde;

                            [         2           ]
                            [   u    c  rho       ]
                            [ -----  ------  0  0 ]
                            [ theta  theta        ]
                            [                     ]
                            [   1                 ]
                            [  ---     u     0  0 ]
(%o51)                      [  rho                ]
                            [                     ]
                            [   0      0     u  0 ]
                            [                     ]
                            [           2         ]
                            [          c          ]
                            [   0      --    0  u ]
                            [          cp         ]

In [28]:
lambdas: eigenvalues(invert(Gamma).Atilde);

                      2                 2      2
           sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + ((- theta) - 1) u
(%o52) [[- ----------------------------------------------------------------, 
                                       2 theta
              2                 2      2
   sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + (theta + 1) u
   ------------------------------------------------------------, u], [1, 1, 2]]
                             2 theta

In [29]:
ratsimp(lambdas[1][1]/lambdas[1][2]);

                    2                 2      2
         sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + ((- theta) - 1) u
(%o53) - ----------------------------------------------------------------
                      2                 2      2
           sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + (theta + 1) u

***
# 4. Preconditioning matrix $\Gamma$

### The above shows how the conversion from conservative to primitive variables work. Note that in the paper of Weiss and Smith, the derivatives of density with respect to temperature and pressure are kept to allow for a general preconditioning matrix that does not depend on a specific equation of state.

### For the equation of state in case of the ideal gas law, note that $P=\rho R T$, and $\rho=\frac{P}{RT}$, so the derivative is:

$$\color{blue}{\large \frac{\partial \rho}{\partial p} = \frac{1}{RT} }$$

### And since $c^2 = \gamma R T$, we have:

$$\color{blue}{\large \frac{\partial \rho}{\partial p} = \frac{\gamma}{c^2} }$$

### For derivative wrt temperarure we have :
    


In [30]:
kill(T);
kill(p);

subst(p=rho*R*T,diff(p/(R*T),T));

(%o54)                               done

(%o55)                               done

                                       rho
(%o56)                               - ---
                                        T

### and the derivative is therefore

$$ \color{blue}{\large \frac{\partial \rho}{\partial T} = -\frac{\rho}{T} }$$

### We can substitute these into the Weiss and Smith matrices to retrieve the result obtained above. We will now first derive the Weiss and Smith result, without using the equation of state (ideal gas law). In this case, the density is simply an unknown function of pressure and temperature:

In [31]:
depends(rho,[p,T]);

(%o57)                            [rho(p, T)]

### Now we compute the transformation matrix $M = \frac{\partial U}{\partial V}$ to go from conservative variables $U$ to primitive variables $V$. This is equation (7) in the paper of Economon, or below eq. 2 in Weiss and Smith:


In [32]:

V:matrix([p,u,v,T]);

E:H - p/rho;
/* note that we can keep the general term dh/dT here*/
gradef(H,T,cp);
gradef(H,u,u);
gradef(H,v,v);
gradef(H,w,w);

U:matrix([rho,rho*u,rho*v,rho*E]);
V;
/* *we directly convert from conservative variables to primitive variables */
dUdV : ratsimp(jacobian(args(U)[1],args(V)[1]));

(%o58)                          [ p  u  v  T ]

                                         p
(%o59)                              H - ---
                                        rho

(%o60)                                 H

(%o61)                                 H

(%o62)                                 H

(%o63)                                 H

                     [                          p       ]
(%o64)               [ rho  rho u  rho v  (H - ---) rho ]
                     [                         rho      ]

(%o65)                          [ p  u  v  T ]

                 [    drho                        drho       ]
                 [    ----       0      0         ----       ]
                 [     dp                          dT        ]
                 [                                           ]
                 [   drho                        drho        ]
                 [   ---- u     rho     0        ---- u      ]
                 [    dp                          dT         ]
(%o66)           [                                           ]
                 [   drho                        drho        ]
                 [   ---- v      0     rho       ---- v      ]
                 [    dp                          dT         ]
                 [                                           ]
                 [   drho                      drho          ]
                 [ H ---- - 1  rho u  rho v  H ---- + cp rho ]
                 [    dp                        dT           ]

### We see that we have recovered the result of Weiss and Smith, matrix $\frac{\partial W}{\partial Q} = \frac{\partial U}{\partial V}$, below equation (2). Note that this should be the same as the matrix M defined above, where the ideal gas law was used to determine $\frac{\partial \rho}{\partial T}$, $\frac{\partial \rho}{\partial p}$ and the enthalpy $H$. Note that for the sensible enthalpy, we have used the simplification that $h = \int c_p dT \approx c_p T$:

In [33]:
M;

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o67)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

### Let's substitute the ideal gas law and check that we indeed have the same matrices. Note that $p=\rho R T$, and $\frac{a^2}{\gamma} = \frac{p}{\rho}$, so:
$$\color{blue}{\large RT = \frac{a^2}{\gamma}}$$

### and with $\frac{c_p}{c_v}=\gamma$ and  $R = c_p - c_v = c_p(1-\frac{1}{\gamma})$:
$$\color{blue}{\large T = \frac{a^2}{\gamma}\frac{1}{c_p(1-\frac{1}{\gamma})},}$$
### and finally:

$$\color{blue}{\large T = \frac{a^2}{ c_p (\gamma - 1)}}$$

In [34]:
dUdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],dUdV);
dUdV2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),dUdV2));

(%o68) 
    [           gamma                                         rho             ]
    [           -----               0      0                - ---             ]
    [             2                                            T              ]
    [            c                                                            ]
    [                                                                         ]
    [          u gamma                                       rho u            ]
    [          -------             rho     0               - -----            ]
    [             2                                            T              ]
    [            c                                                            ]
    [                                                                         ]
    [          v gamma                                       rho v            ]
    [          -------              0     rho              - -----            ]
    [             2             

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o69)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

### Notice that using the two different routes, we obtain the same result, as $\frac{\partial U}{\partial V} = M$
### Let's keep working for now with the independent (from the equation of state) transformation matrix: 

### Note that the flux is $\frac{\partial F(V)}{\partial x}$, with $V=(p,u,v,T)$ and because we have defined the derivative of density with respect to pressure and temperarure, we can obtain the transformation into quasilinear form directly using:

$$\color{blue}{ \large \frac{\partial F(V)}{\partial x} = \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}}$$

In [35]:
F:ratsimp((matrix([rho*u,rho*u*u+p,rho*u*v,u*rho*E+p*u])));

(%o70)              [             2                       ]
                    [ rho u  rho u  + p  rho u v  H rho u ]

### and its Jacobian with respect to $V=(\rho, u,v, T)$ can be determined directly (because we also have the derivative of H(T) defined):

In [36]:
dFdV : jacobian(args(F)[1],args(V)[1]);

         [   drho                                      drho          ]
         [   ---- u          rho           0           ---- u        ]
         [    dp                                        dT           ]
         [                                                           ]
         [ drho  2                                     drho  2       ]
         [ ---- u  + 1     2 rho u         0           ---- u        ]
         [  dp                                          dT           ]
(%o71)   [                                                           ]
         [  drho                                      drho           ]
         [  ---- u v        rho v        rho u        ---- u v       ]
         [   dp                                        dT            ]
         [                                                           ]
         [    drho           2                     drho              ]
         [  H ---- u    rho u  + H rho  rho u v  H ---- u + cp rho u ]
      

### The above Jacobian matrix is independent of the equation of state. We can substitute the ideal gas law to get to the matrix that is valid only for an ideal gas:

In [37]:
dFdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],dFdV);
dFdV2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),dFdV2));

                [         u gamma          ]
                [         -------          ]
                [            2             ]
                [           c              ]
                [                          ]
                [        2                 ]
                [       u  gamma           ]
                [       -------- + 1       ]
                [           2              ]
                [          c               ]
                [                          ]
(%o72)  Col 1 = [        u v gamma         ]
                [        ---------         ]
                [            2             ]
                [           c              ]
                [                          ]
                [     2    2               ]
                [    v  + u                ]
                [ u (------- + T cp) gamma ]
                [       2                  ]
                [ ------------------------ ]
                [             2            ]
          

                [                       u gamma                       ]
                [                       -------                       ]
                [                          2                          ]
                [                         c                           ]
                [                                                     ]
                [                     2          2                    ]
                [                    u  gamma + c                     ]
                [                    -------------                    ]
                [                          2                          ]
                [                         c                           ]
(%o73)  Col 1 = [                                                     ]
                [                      u v gamma                      ]
                [                      ---------                      ]
                [                          2                    

### We can now write the pde in primitive variables $V$ as:
$$\color{blue}{\large \frac{\partial U}{\partial t} + \frac{\partial F}{\partial x}=0}$$


$$\color{blue}{\large \frac{\partial U}{\partial V} \frac{\partial V}{\partial t} + \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0}$$


$$\color{blue}{\large \frac{\partial V}{\partial t} + [\frac{\partial U}{\partial V}]^{-1} \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0}$$



### We us the matrices with the ideal gas law as the equation of state: 

In [38]:
ratsimp(invert(dUdV2));

                [             2    2           2    2           ]
                [           (v  + u ) gamma - v  - u            ]
                [           -------------------------           ]
                [                       2                       ]
                [                                               ]
                [                        u                      ]
                [                     - ---                     ]
                [                       rho                     ]
(%o74)  Col 1 = [                                               ]
                [                        v                      ]
                [                     - ---                     ]
                [                       rho                     ]
                [                                               ]
                [   2    2       2        2     2             2 ]
                [ (v  + u ) gamma  + ((- v ) - u ) gamma - 2 c  ]
          

In [39]:
Atilde : ratsimp(invert(dUdV2).dFdV2);

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o75)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

In [40]:
eigenvalues(Atilde);

(%o76)                  [[u - c, u + c, u], [1, 1, 2]]

### ...And we see again that we retrieve the same matrix...
### We can do the same for the flux in y-direction:

In [41]:
G:ratsimp((matrix([rho*v,rho*v*u,rho*v*v+p,v*rho*E+p*v])));
dGdV : jacobian(args(G)[1],args(V)[1]);
dGdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],dGdV);
dGdV2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),dGdV2));
Btilde : ratsimp(invert(dUdV2).dGdV2);
eigenvalues(Btilde);

(%o77)              [                      2              ]
                    [ rho v  rho u v  rho v  + p  H rho v ]

         [   drho                                      drho          ]
         [   ---- v        0          rho              ---- v        ]
         [    dp                                        dT           ]
         [                                                           ]
         [  drho                                      drho           ]
         [  ---- u v     rho v       rho u            ---- u v       ]
         [   dp                                        dT            ]
(%o78)   [                                                           ]
         [ drho  2                                     drho  2       ]
         [ ---- v  + 1     0        2 rho v            ---- v        ]
         [  dp                                          dT           ]
         [                                                           ]
         [    drho                    2            drho              ]
         [  H ---- v    rho u v  rho v  + H rho  H ---- v + cp rho v ]
      

                [         v gamma          ]
                [         -------          ]
                [            2             ]
                [           c              ]
                [                          ]
                [        u v gamma         ]
                [        ---------         ]
                [            2             ]
                [           c              ]         [    0    ]
                [                          ]         [         ]
                [        2                 ]         [  rho v  ]
(%o79)  Col 1 = [       v  gamma           ] Col 2 = [         ]
                [       -------- + 1       ]         [    0    ]
                [           2              ]         [         ]
                [          c               ]         [ rho u v ]
                [                          ]
                [     2    2               ]
                [    v  + u                ]
                [ v (------- + T cp) gamma ]
     

                [                     v gamma                     ]
                [                     -------                     ]
                [                        2                        ]
                [                       c                         ]
                [                                                 ]
                [                    u v gamma                    ]
                [                    ---------                    ]
                [                        2                        ]
                [                       c                         ]
                [                                                 ]
(%o80)  Col 1 = [                   2          2                  ]
                [                  v  gamma + c                   ]
                [                  -------------                  ]
                [                        2                        ]
                [                       c       

                             [          2        ]
                             [  v   0  c  rho  0 ]
                             [                   ]
                             [  0   v    0     0 ]
                             [                   ]
                             [  1                ]
(%o81)                       [ ---  0    v     0 ]
                             [ rho               ]
                             [                   ]
                             [            2      ]
                             [           c       ]
                             [  0   0    --    v ]
                             [           cp      ]

(%o82)                  [[v - c, v + c, v], [1, 1, 2]]

***
# 5. Flux Jacobian
### The flux Jacobian matrix of the combined system is $A_c = A_x\cdot n_x + A_y\cdot n_y$ :

In [42]:
Ac : ratsimp(Atilde*n_x + Btilde*n_y);

        [                  2              2                          ]
        [ n_y v + n_x u   c  n_x rho     c  n_y rho          0       ]
        [                                                            ]
        [      n_x                                                   ]
        [      ---       n_y v + n_x u        0              0       ]
        [      rho                                                   ]
        [                                                            ]
(%o83)  [      n_y                                                   ]
        [      ---             0        n_y v + n_x u        0       ]
        [      rho                                                   ]
        [                                                            ]
        [                    2              2                        ]
        [                   c  n_x         c  n_y                    ]
        [       0           ------         ------      n_y v + n_x u ]
      

In [43]:
eigenvalues(Ac);

                                   2      2
(%o84) [[n_y v + n_x u - c sqrt(n_y  + n_x ), 
                                          2      2
                n_y v + n_x u + c sqrt(n_y  + n_x ), n_y v + n_x u], [1, 1, 2]]

In [45]:
eigenvectors(Ac)[2];

                           2      2                 2      2
               n_x sqrt(n_y  + n_x )    n_y sqrt(n_y  + n_x )    1
(%o86) [[[1, - ---------------------, - ---------------------, ------]], 
                     2        2               2        2       cp rho
               (c n_y  + c n_x ) rho    (c n_y  + c n_x ) rho
                 2      2               2      2
     n_x sqrt(n_y  + n_x )  n_y sqrt(n_y  + n_x )    1
[[1, ---------------------, ---------------------, ------]], 
           2        2             2        2       cp rho
     (c n_y  + c n_x ) rho  (c n_y  + c n_x ) rho
          n_x
[[0, 1, - ---, 0], [0, 0, 0, 1]]]
          n_y

### We now want to introduce a matrix $\Gamma$ that premultiplies the temporal derivative. We use as a starting point the previously derived PDE:

$$\color{blue}{\large \frac{\partial U}{\partial t} + \frac{\partial F(U)}{\partial x}=0}$$


$$\color{blue}{\large \frac{\partial U}{\partial V} \frac{\partial V}{\partial t} + \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0}$$

### Note that we can modify the Jacobian matrix $\frac{\partial U}{\partial V}$ without affecting the steady state solution. We replace it with a preconditioning matrix $\Gamma:$

$$\color{blue}{\large \Gamma \frac{\partial V}{\partial t} + \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0}$$

$$\color{blue}{\large \frac{\partial V}{\partial t} + \Gamma^{-1} \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0}$$


### We now introduce a preconditioning matrix and see whether we can modify the eigenvalues in a beneficial way:
### We use the Weiss and Smith approach that $\frac{\partial \rho}{\partial p} = \theta$

In [63]:
dUdV;

                 [    drho                        drho       ]
                 [    ----       0      0         ----       ]
                 [     dp                          dT        ]
                 [                                           ]
                 [   drho                        drho        ]
                 [   ---- u     rho     0        ---- u      ]
                 [    dp                          dT         ]
(%o124)          [                                           ]
                 [   drho                        drho        ]
                 [   ---- v      0     rho       ---- v      ]
                 [    dp                          dT         ]
                 [                                           ]
                 [   drho                      drho          ]
                 [ H ---- - 1  rho u  rho v  H ---- + cp rho ]
                 [    dp                        dT           ]

### We recover the matrix of Weiss and Smith, eq. (4)

### We replace the term $\frac{\partial \rho}{\partial p} = \theta$

In [64]:
Gamma : subst(diff(rho,p)=theta,dUdV);

                [                                 drho       ]
                [    theta       0      0         ----       ]
                [                                  dT        ]
                [                                            ]
                [                                drho        ]
                [   theta u     rho     0        ---- u      ]
                [                                 dT         ]
(%o125)         [                                            ]
                [                                drho        ]
                [   theta v      0     rho       ---- v      ]
                [                                 dT         ]
                [                                            ]
                [                              drho          ]
                [ H theta - 1  rho u  rho v  H ---- + cp rho ]
                [                               dT           ]

In [65]:
invGamma : ratsimp(invert(Gamma));

                 [   drho  2   drho  2     drho          ]
                 [   ---- v  + ---- u  - H ---- - cp rho ]
                 [    dT        dT          dT           ]
                 [ - ----------------------------------- ]
                 [                          drho         ]
                 [           cp rho theta + ----         ]
                 [                           dT          ]
                 [                                       ]
                 [                    u                  ]
                 [                 - ---                 ]
(%o126)  Col 1 = [                   rho                 ]
                 [                                       ]
                 [                    v                  ]
                 [                 - ---                 ]
                 [                   rho                 ]
                 [                                       ]
                 [          2          2                

In [72]:
Ac : ratsimp(invGamma.dFdV);

             [         drho   drho                               ]
             [ (cp rho ---- + ----) u              2             ]
             [          dp     dT            cp rho              ]
             [ ----------------------  -------------------  0  0 ]
             [                 drho                   drho       ]
             [  cp rho theta + ----    cp rho theta + ----       ]
             [                  dT                     dT        ]
             [                                                   ]
             [           1                                       ]
             [          ---                     u           0  0 ]
(%o133)      [          rho                                      ]
             [                                                   ]
             [           0                      0           u  0 ]
             [                                                   ]
             [             drho                               

In [80]:
Bc : ratsimp(invGamma.dGdV);

             [         drho   drho                               ]
             [ (cp rho ---- + ----) v                 2          ]
             [          dp     dT               cp rho           ]
             [ ----------------------  0  -------------------  0 ]
             [                 drho                      drho    ]
             [  cp rho theta + ----       cp rho theta + ----    ]
             [                  dT                        dT     ]
             [                                                   ]
             [           0             v           0           0 ]
             [                                                   ]
(%o148)      [           1                                       ]
             [          ---            0           v           0 ]
             [          rho                                      ]
             [                                                   ]
             [             drho                               

### Now with substitution of the ideal gas law:

In [78]:
Ac_1 : subst([diff(rho,p)=gamma/c^2],Ac);
Ac_2 : ratsimp(subst([diff(rho,T)=-rho*cp*(gamma-1)/c^2],Ac_1));
ratexpand(eigenvalues(Ac_2));

            [    cp rho gamma   drho                             ]
            [ u (------------ + ----)                            ]
            [          2         dT                2             ]
            [         c                      cp rho              ]
            [ -----------------------  -------------------  0  0 ]
            [                  drho                   drho       ]
            [   cp rho theta + ----    cp rho theta + ----       ]
            [                   dT                     dT        ]
            [                                                    ]
            [            1                                       ]
            [           ---                     u           0  0 ]
(%o144)     [           rho                                      ]
            [                                                    ]
            [            0                      0           u  0 ]
            [                                                 

                 [                     u                     ]
                 [          - --------------------           ]
                 [                     2                     ]
                 [            gamma - c  theta - 1           ]
                 [                                           ]
                 [                     1                     ]
                 [                    ---                    ]
(%o145)  Col 1 = [                    rho                    ]
                 [                                           ]
                 [                     0                     ]
                 [                                           ]
                 [                       2                   ]
                 [            u gamma - c  theta u           ]
                 [ - --------------------------------------- ]
                 [                   2                       ]
                 [   cp rho gamma - c  cp rho theta - c

                2      2          2        2       2           4      2  2
(%o146) [[sqrt(u  gamma  + ((- 2 c  theta u ) - 4 c ) gamma + c  theta  u
      4            2                2                      u gamma
 + 4 c  theta + 4 c )/(2 gamma - 2 c  theta - 2) + ------------------------
                                                                2
                                                   2 gamma - 2 c  theta - 2
           2
          c  theta u                   u
 - ------------------------ - --------------------, 
                2                      2
   2 gamma - 2 c  theta - 2   gamma - c  theta - 1
         2      2          2        2       2           4      2  2
(- sqrt(u  gamma  + ((- 2 c  theta u ) - 4 c ) gamma + c  theta  u
      4            2                2                       u gamma
 + 4 c  theta + 4 c )/(2 gamma - 2 c  theta - 2)) + ------------------------
                                                                 2
                    

In [72]:
eigenvectors(Ac);

                      2    2      2       2    2 drho           2    2  drho 2
(%o125) [[[- (sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) )
                                                  dp                     dp
  2       2    2                  drho
 u  + 4 cp  rho  theta + 4 cp rho ----)
                                   dT
                              drho     drho                         drho
 + ((- cp rho theta) - cp rho ---- - 2 ----) u)/(2 cp rho theta + 2 ----), 
                               dp       dT                           dT
         2    2      2       2    2 drho           2    2  drho 2   2
(sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                     dp                     dp
       2    2                  drho                           drho     drho
 + 4 cp  rho  theta + 4 cp rho ----) + (cp rho theta + cp rho ---- + 2 ----) u)
                                dT                             dp    

### let's store the eigenvalues:

In [73]:
lambdas : ratsimp(eigenvalues(ratsimp(invGamma.dFdV)));

                     2    2      2       2    2 drho           2    2  drho 2
(%o126) [[- (sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) )
                                                 dp                     dp
  2       2    2                  drho
 u  + 4 cp  rho  theta + 4 cp rho ----)
                                   dT
                              drho     drho                         drho
 + ((- cp rho theta) - cp rho ---- - 2 ----) u)/(2 cp rho theta + 2 ----), 
                               dp       dT                           dT
         2    2      2       2    2 drho           2    2  drho 2   2
(sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                     dp                     dp
       2    2                  drho                           drho     drho
 + 4 cp  rho  theta + 4 cp rho ----) + (cp rho theta + cp rho ---- + 2 ----) u)
                                dT                             dp       

### What we now have to show is that these eigenvalues are prefered.

In [74]:
l1 : lambdas[1][1];

                   2    2      2       2    2 drho           2    2  drho 2   2
(%o127) - (sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                               dp                     dp
       2    2                  drho                               drho
 + 4 cp  rho  theta + 4 cp rho ----) + ((- cp rho theta) - cp rho ----
                                dT                                 dp
     drho                         drho
 - 2 ----) u)/(2 cp rho theta + 2 ----)
      dT                           dT

In [100]:
l2 : lambdas[1][2];

                 2    2      2       2    2 drho           2    2  drho 2   2
(%o157) (sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                             dp                     dp
       2    2                  drho                           drho     drho
 + 4 cp  rho  theta + 4 cp rho ----) + (cp rho theta + cp rho ---- + 2 ----) u)
                                dT                             dp       dT
                     drho
/(2 cp rho theta + 2 ----)
                      dT

In [101]:
ratexpand(l1);

                   2    2      2       2    2 drho           2    2  drho 2   2
(%o158) (- sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                               dp                     dp
       2    2                  drho                      drho
 + 4 cp  rho  theta + 4 cp rho ----)/(2 cp rho theta + 2 ----))
                                dT                        dT
                                         drho                drho
                                  cp rho ---- u              ---- u
       cp rho theta u                     dp                  dT
 + ----------------------- + ----------------------- + -------------------
                      drho                      drho                  drho
   2 cp rho theta + 2 ----   2 cp rho theta + 2 ----   cp rho theta + ----
                       dT                        dT                    dT

In [102]:
l1_1 : ratexpand(ratsimp(subst(theta=1/ur^2 - diff(rho,T)/(rho*cp),l1)));

                   2    2  drho 2            drho drho    drho 2   2   4
(%o159) (- sqrt((cp  rho  (----)  + 2 cp rho ---- ---- + (----) ) u  ur
                            dp                dT   dp      dT
            2    2 drho             drho   2       2    2    2     2    2  2
 + (((- 2 cp  rho  ----) - 2 cp rho ----) u  + 4 cp  rho ) ur  + cp  rho  u )
                    dp               dT
               drho     2   drho     2
               ---- u ur    ---- u ur
                dp           dT          u
/(2 cp rho)) + ---------- + ---------- + -
                   2         2 cp rho    2

In [103]:
l2_1 : ratexpand(ratsimp(subst(theta=1/ur^2 - diff(rho,T)/(rho*cp),l2)));

                2    2  drho 2            drho drho    drho 2   2   4
(%o160) sqrt((cp  rho  (----)  + 2 cp rho ---- ---- + (----) ) u  ur
                         dp                dT   dp      dT
            2    2 drho             drho   2       2    2    2     2    2  2
 + (((- 2 cp  rho  ----) - 2 cp rho ----) u  + 4 cp  rho ) ur  + cp  rho  u )
                    dp               dT
              drho     2   drho     2
              ---- u ur    ---- u ur
               dp           dT          u
/(2 cp rho) + ---------- + ---------- + -
                  2         2 cp rho    2

In [104]:
beta : diff(rho,p)+diff(rho,T)/(rho*cp);

alpha : (1-beta*ur^2)/2;

up : u*(1-alpha);
ratsimp(up);
ratexpand(ratsimp(up));

                                         drho
                                         ----
                                 drho     dT
(%o161)                          ---- + ------
                                  dp    cp rho

                                         drho
                                         ----
                                 drho     dT      2
                            1 - (---- + ------) ur
                                  dp    cp rho
(%o162)                     -----------------------
                                       2

                                            drho
                                            ----
                                    drho     dT      2
                               1 - (---- + ------) ur
                                     dp    cp rho
(%o163)                 u (1 - -----------------------)
                                          2

                             drho   drho      2
                     (cp rho ---- + ----) u ur  + cp rho u
                              dp     dT
(%o164)              -------------------------------------
                                   2 cp rho

                          drho     2   drho     2
                          ---- u ur    ---- u ur
                           dp           dT          u
(%o165)                   ---------- + ---------- + -
                              2         2 cp rho    2

In [105]:
l1_2 : ratsimp(subst(cp = diff(rho,T)/(rho*(betap-diff(rho,p))),l1_1));
l1_3 : ratexpand(ratsimp(subst(betap=(1-2*alphap)/ur^2,l1_2)));

          drho                    2  drho 2  2   4
(%o166) ((---- - betap) sqrt(betap  (----)  u  ur
           dp                         dT
       drho 2            drho 2  2    2    drho 2  2
 + (4 (----)  - 2 betap (----)  u ) ur  + (----)  u )
        dT                dT                dT
         drho 2           drho        2         drho     2   drho
 + sqrt((----)  - 2 betap ---- + betap ) (betap ---- u ur  + ---- u))
          dp               dp                    dT           dT
    drho       drho 2           drho        2
/(2 ---- sqrt((----)  - 2 betap ---- + betap ))
     dT         dp               dp

(%o167) 
             drho   2         drho 2   2           2  drho 2  2
             ---- ur  sqrt(4 (----)  ur  + 4 alphap  (----)  u )
              dp               dT                      dT
-----------------------------------------------------------------------------
  drho       drho 2   4                  drho   2           2
2 ---- sqrt((----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap + 1)
   dT         dp                          dp
                               drho 2   2           2  drho 2  2
                alphap sqrt(4 (----)  ur  + 4 alphap  (----)  u )
                                dT                      dT
 + ---------------------------------------------------------------------------
   drho       drho 2   4                  drho   2           2
   ---- sqrt((----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap + 1)
    dT         dp                          dp
           drho 2   2           2  drho 2  2
 - sqrt(4 (----)  ur  + 4 alphap  (----

In [106]:
l2_2 : ratsimp(subst(cp = diff(rho,T)/(rho*(betap-diff(rho,p))),l2_1));
l2_3 : ratexpand(ratsimp(subst(betap=(1-2*alphap)/ur^2,l2_2)));

            drho                    2  drho 2  2   4
(%o168) - ((---- - betap) sqrt(betap  (----)  u  ur
             dp                         dT
       drho 2            drho 2  2    2    drho 2  2
 + (4 (----)  - 2 betap (----)  u ) ur  + (----)  u )
        dT                dT                dT
         drho 2           drho        2            drho     2    drho
 + sqrt((----)  - 2 betap ---- + betap ) ((- betap ---- u ur ) - ---- u))
          dp               dp                       dT            dT
    drho       drho 2           drho        2
/(2 ---- sqrt((----)  - 2 betap ---- + betap ))
     dT         dp               dp

            drho   2         drho 2   2           2  drho 2  2
(%o169) (- (---- ur  sqrt(4 (----)  ur  + 4 alphap  (----)  u ))
             dp               dT                      dT
    drho       drho 2   4                  drho   2           2
/(2 ---- sqrt((----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap
     dT         dp                          dp
 + 1)))
                               drho 2   2           2  drho 2  2
                alphap sqrt(4 (----)  ur  + 4 alphap  (----)  u )
                                dT                      dT
 - ---------------------------------------------------------------------------
   drho       drho 2   4                  drho   2           2
   ---- sqrt((----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap + 1)
    dT         dp                          dp
           drho 2   2           2  drho 2  2
 + sqrt(4 (----)  ur  + 4 alphap  (----)  u )
            dT                      dT
    drho       drho 2   4         

### We see that the part that does not involve the root is simply $u(1-\alpha)$. We remove it from the expression to further simplify the root.

In [107]:
l1_4 : rootscontract(ratsimp(l1_3 +alphap*u-u));

          drho   2
(%o170) ((---- ur  + 2 alphap - 1)
           dp
                       drho 2   2           2  drho 2  2
                    4 (----)  ur  + 4 alphap  (----)  u
                        dT                      dT
 sqrt(----------------------------------------------------------------))
       drho 2   4                  drho   2           2
      (----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap + 1
        dp                          dp
    drho
/(2 ----)
     dT

In [109]:
l2_4 : rootscontract(ratsimp(l2_3 +alphap*u-u));

            drho   2
(%o172) - ((---- ur  + 2 alphap - 1)
             dp
                       drho 2   2           2  drho 2  2
                    4 (----)  ur  + 4 alphap  (----)  u
                        dT                      dT
 sqrt(----------------------------------------------------------------))
       drho 2   4                  drho   2           2
      (----)  ur  + (4 alphap - 2) ---- ur  + 4 alphap  - 4 alphap + 1
        dp                          dp
    drho
/(2 ----)
     dT

### multiply to get rid of the root, this will make simplification easier

In [111]:
l1_5 : sqrt(ratsimp(l1_4*l1_4));
l2_5 : sqrt(ratsimp(l2_4*l2_4));

                                   2         2  2
(%o175)                     sqrt(ur  + alphap  u )

                                   2         2  2
(%o176)                     sqrt(ur  + alphap  u )

### and we have recovered the result of Weiss and Smith, eq. (11), where the eigenvalue is $u' + c'$. What remains is to show that these eigenvalues are beneficial. We see that as $U_r\rightarrow 0$, then $\alpha^{'} \rightarrow \frac{1}{2}$. In that case we have for the eigenvalue:

$$\color{blue}{\large u^{'} + c^{'} = \frac{1}{2} u + \frac{1}{2}u = u}$$

### Note also that we can simplify the expression for $\theta$ when we are not interested in retaining proper behavior when $u \rightarrow c$. We can simply use $\theta=\frac{1}{U_r^2}$, and when using the ideal gas law, this results in: 

In [83]:
l1;

                   2    2      2       2    2 drho           2    2  drho 2   2
(%o141) - (sqrt((cp  rho  theta  - 2 cp  rho  ---- theta + cp  rho  (----) ) u
                                               dp                     dp
       2    2                  drho                               drho
 + 4 cp  rho  theta + 4 cp rho ----) + ((- cp rho theta) - cp rho ----
                                dT                                 dp
     drho                         drho
 - 2 ----) u)/(2 cp rho theta + 2 ----)
      dT                           dT

In [82]:
l1_1 : ratexpand(ratsimp(subst([theta=1/ur^2, diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2],l1)));

                     2    2  2   4      2        2   2    2  2   2
(%o140) (T sqrt((T cp  rho  u  ur  gamma  - 2 T c  cp  rho  u  ur  gamma
      4       2   4        4   2    2   2      4   2    2  2
 - 4 c  cp rho  ur  + 4 T c  cp  rho  ur  + T c  cp  rho  u )/T))
                                             2
     2       2        2             T cp u ur  gamma         T cp u
/(2 c  rho ur  - 2 T c  cp rho) - -------------------- - --------------
                                     2   2        2          2
                                  2 c  ur  - 2 T c  cp   2 ur  - 2 T cp
         2
     u ur
 + ----------
     2
   ur  - T cp

### We split it into a part containing the square root and the second part:

In [84]:
grind(l1_1);

(%o142)                              done


(T*sqrt((T*cp^2*rho^2*u^2*ur^4*gamma^2-2*T*c^2*cp^2*rho^2*u^2*ur^2*gamma
                                      -4*c^4*cp*rho^2*ur^4
                                      +4*T*c^4*cp^2*rho^2*ur^2
                                      +T*c^4*cp^2*rho^2*u^2)
         /T))
 /(2*c^2*rho*ur^2-2*T*c^2*cp*rho)
 -(T*cp*u*ur^2*gamma)/(2*c^2*ur^2-2*T*c^2*cp)-(T*cp*u)/(2*ur^2-2*T*cp)
 +(u*ur^2)/(ur^2-T*cp)$


In [116]:
p: factor(ratsimp(-(T*cp*u*ur^2*gamma)/(2*c^2*ur^2-2*T*c^2*cp)-(T*cp*u)/(2*ur^2-2*T*cp)
 +(u*ur^2)/(ur^2-T*cp)));
p:subst(T=c*c/(gamma*R),p);
p:subst(R=cp/cv,p);
p:factor(ratsimp(subst(cv=cp/gamma,p)));

                               2            2   2      2
                     u (T cp ur  gamma - 2 c  ur  + T c  cp)
(%o184)            - ---------------------------------------
                                   2    2
                                2 c  (ur  - T cp)

                            4        2      2
                           c  cp    c  cp ur       2   2
                       u (------- + --------- - 2 c  ur )
                          R gamma       R
(%o185)              - ----------------------------------
                                            2
                                 2    2    c  cp
                              2 c  (ur  - -------)
                                          R gamma

                            4
                           c  cv    2      2      2   2
                        u (----- + c  cv ur  - 2 c  ur )
                           gamma
(%o186)               - --------------------------------
                                            2
                                  2    2   c  cv
                               2 c  (ur  - -----)
                                           gamma

                           2      2        2          2
                    u (2 ur  gamma  - cp ur  gamma - c  cp)
(%o187)             ---------------------------------------
                                 2      2    2
                            2 (ur  gamma  - c  cp)

In [118]:
l1_2:(T*sqrt((T*cp^2*rho^2*u^2*ur^4*gamma^2-2*T*c^2*cp^2*rho^2*u^2*ur^2*gamma
                                      -4*c^4*cp*rho^2*ur^4
                                      +4*T*c^4*cp^2*rho^2*ur^2
                                      +T*c^4*cp^2*rho^2*u^2)
         /T))
 /(2*c^2*rho*ur^2-2*T*c^2*cp*rho);

                     2    2  2   4      2        2   2    2  2   2
(%o191) (T sqrt((T cp  rho  u  ur  gamma  - 2 T c  cp  rho  u  ur  gamma
      4       2   4        4   2    2   2      4   2    2  2
 - 4 c  cp rho  ur  + 4 T c  cp  rho  ur  + T c  cp  rho  u )/T))
     2       2        2
/(2 c  rho ur  - 2 T c  cp rho)

In [122]:
l1_3:factor(ratsimp(l1_2*l1_2));
l1_4 : (ratsimp(subst(T=c*c/((cp-cv)*gamma),l1_3)));
l1_5:factor(ratsimp(subst(cv=cp/gamma,l1_4)));

                     2   4      2        2     2   2            4   4
(%o201) (T cp (T cp u  ur  gamma  - 2 T c  cp u  ur  gamma - 4 c  ur
                                   4      2      4     2       4    2        2
                            + 4 T c  cp ur  + T c  cp u ))/(4 c  (ur  - T cp) )

           2  2   4      2        2            2   2    4      2   2  2   2
(%o202) (cp  u  ur  gamma  + ((4 c  cp cv - 4 c  cp ) ur  - 2 c  cp  u  ur )
            4   2   2    4   2  2        2                 2    4      2
 gamma + 4 c  cp  ur  + c  cp  u )/((4 cv  - 8 cp cv + 4 cp ) ur  gamma
       2            2   2    2            4   2
 + (8 c  cp cv - 8 c  cp ) ur  gamma + 4 c  cp )

          2   4      2      2   4            2  2   2            2   4
(%o203) (u  ur  gamma  - 4 c  ur  gamma - 2 c  u  ur  gamma + 4 c  ur
                                     4   2    4  2        2           2    2 2
                                + 4 c  ur  + c  u )/(4 (ur  gamma - ur  - c ) )

In [131]:
l1_6:subst(u=a*ur,l1_5);

          2   6      2      2  2   4            2   4            2   4
(%o211) (a  ur  gamma  - 2 a  c  ur  gamma - 4 c  ur  gamma + 4 c  ur
                               2  4   2      4   2        2           2    2 2
                            + a  c  ur  + 4 c  ur )/(4 (ur  gamma - ur  - c ) )

In [134]:
l1_7:subst(ur=m*c,l1_6);

          2  6  6      2      2  6  4            6  4            6  4
(%o214) (a  c  m  gamma  - 2 a  c  m  gamma - 4 c  m  gamma + 4 c  m
                             2  6  2      6  2       2  2          2  2    2 2
                          + a  c  m  + 4 c  m )/(4 (c  m  gamma - c  m  - c ) )

# Preconditioned Flux difference Splitting
### We now derive the necessary matrices for the flux difference splitting approach in 2D. The nonpreconditioned flux difference splitting is given by:

$$\color{blue}{\large \frac{1}{2}(F_i + F_j)\cdot\overline n_{ij} -\frac{1}{2}|\frac{\partial F}{\partial U}|(U_i-U_j) }$$

$$\color{blue}{\large \frac{1}{2}(F_i + F_j)\cdot\overline n_{ij} -\frac{1}{2}\Gamma |A_{\Gamma}|(U_i-U_j) }$$

$$\color{blue}{\large \frac{1}{2}(F_i + F_j)\cdot\overline n_{ij} -\frac{1}{2}\Gamma P |\Lambda_{\Gamma}|P^{-1}(U_i-U_j),}$$

### So for the matrices $P$ and $P^{-1}$ we need the eigenvectors of the preconditioned jacobian matrix $A_{\Gamma}$.

In [81]:
Ac;

             [         drho   drho                               ]
             [ (cp rho ---- + ----) u              2             ]
             [          dp     dT            cp rho              ]
             [ ----------------------  -------------------  0  0 ]
             [                 drho                   drho       ]
             [  cp rho theta + ----    cp rho theta + ----       ]
             [                  dT                     dT        ]
             [                                                   ]
             [           1                                       ]
             [          ---                     u           0  0 ]
(%o149)      [          rho                                      ]
             [                                                   ]
             [           0                      0           u  0 ]
             [                                                   ]
             [             drho                               

In [82]:
Bc;

             [         drho   drho                               ]
             [ (cp rho ---- + ----) v                 2          ]
             [          dp     dT               cp rho           ]
             [ ----------------------  0  -------------------  0 ]
             [                 drho                      drho    ]
             [  cp rho theta + ----       cp rho theta + ----    ]
             [                  dT                        dT     ]
             [                                                   ]
             [           0             v           0           0 ]
             [                                                   ]
(%o150)      [           1                                       ]
             [          ---            0           v           0 ]
             [          rho                                      ]
             [                                                   ]
             [             drho                               

In [84]:
A : ratsimp(Ac*nx+Bc*ny);

(%o152) 
         [            drho      drho                 drho      drho    ]
         [ (cp ny rho ---- + ny ----) v + (cp nx rho ---- + nx ----) u ]
         [             dp        dT                   dp        dT     ]
         [ ----------------------------------------------------------- ]
         [                                    drho                     ]
         [                     cp rho theta + ----                     ]
         [                                     dT                      ]
         [                                                             ]
         [                             nx                              ]
         [                             ---                             ]
         [                             rho                             ]
 Col 1 = [                                                             ]
         [                             ny                              ]
         [                             ---

In [86]:
lambda : eigenvalues(A);

                     2   2    2      2       2   2    2 drho
(%o154) [[- (sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                         dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                                    drho        drho
 + ((- cp n

In [87]:
lambda[1][1];

                   2   2    2      2       2   2    2 drho
(%o155) - (sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                       dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                                    drho        drho
 + ((- cp ny rho 

In [88]:
V : eigenvectors(A);

                      2   2    2      2       2   2    2 drho
(%o156) [[[- (sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                          dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                                    drho        drho
 + ((- c

In [89]:
V[1];

                     2   2    2      2       2   2    2 drho
(%o157) [[- (sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                         dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                                    drho        drho
 + ((- cp n

In [90]:
V[2];

                            2   2    2      2       2   2    2 drho
(%o158) [[[1, - (nx sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                                dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                 drho
 + (cp nx ny rho ----

In [96]:
eV: V[2];

                            2   2    2      2       2   2    2 drho
(%o164) [[[1, - (nx sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                                dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                 drho
 + (cp nx ny rho ----

In [97]:
length(eV);

(%o165)                                3

In [105]:
ratsimp(eV[1][1][1]);
ratsimp(eV[1][1][2]);
ratsimp(eV[1][1][3]);
ratsimp(eV[1][1][4]);

(%o176)                                1

                      2   2    2      2       2   2    2 drho
(%o177) - (nx sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                          dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
                 drho
 + (cp nx ny rho ---- - cp nx ny rho th

                      2   2    2      2       2   2    2 drho
(%o178) - (ny sqrt((cp  ny  rho  theta  - 2 cp  ny  rho  ---- theta
                                                          dp
     2   2    2  drho 2   2        2          2      2
 + cp  ny  rho  (----) ) v  + (2 cp  nx ny rho  theta
                  dp
       2          2 drho             2          2  drho 2
 - 4 cp  nx ny rho  ---- theta + 2 cp  nx ny rho  (----) ) u v
                     dp                             dp
      2   2    2      2       2   2    2 drho           2   2    2  drho 2   2
 + (cp  nx  rho  theta  - 2 cp  nx  rho  ---- theta + cp  nx  rho  (----) ) u
                                          dp                         dp
        2   2       2   2     2                 2          2      drho
 + (4 cp  ny  + 4 cp  nx ) rho  theta + (4 cp ny  + 4 cp nx ) rho ----)
                                                                   dT
         2     drho        2
 + (cp ny  rho ---- - cp ny  rho

                                      1
(%o179)                             ------
                                    cp rho